<a href="https://colab.research.google.com/github/jamestheengineer/data-science-from-scratch-Python/blob/master/Chapter_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!git clone https://github.com/jamestheengineer/data-science-from-scratch-Python.git
%cd data-science-from-scratch-Python/
!pip install import-ipynb
import import_ipynb

Cloning into 'data-science-from-scratch-Python'...
remote: Enumerating objects: 207, done.
remote: Counting objects: 100% (207/207), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 594 (delta 130), reused 3 (delta 1), pack-reused 387
Receiving objects: 100% (594/594), 1.68 MiB | 4.02 MiB/s, done.
Resolving deltas: 100% (370/370), done.
/content/data-science-from-scratch-Python
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=d29ac08d47f411de2e3066b091648173dcbcdfb97c6a7ddb0be1982781de894c
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [10]:
from typing import NamedTuple

class User(NamedTuple):
  id: int
  name: str

users = [User(0, "Hero"), User(1, "Dunn"), User(2, "Sue"), User(3, "Chi"),
         User(4, "Thor"), User(5, "Clive"), User(6, "Hicks"),
         User(7, "Devin"), User(8, "Kate"), User(9, "Klein")]

friend_pairs = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
                (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

from typing import Dict, List

# type alias for keeping track ofr Friendships
Friendships = Dict[int, List[int]]

friendships: Friendships = {user.id: [] for user in users}

for i, j in friend_pairs:
  friendships[i].append(j)
  friendships[j].append(i)

assert friendships[4] == [3, 5]
assert friendships[8] == [6,7,9]

In [11]:
from collections import deque

Path = List[int]

def shortest_paths_from(from_user_id: int,
                        friendships: Friendships) -> Dict[int, List[Path]]:
    # A dictionary from user_id to *all* shortest paths to that user.
    shortest_paths_to: Dict[int, List[Path]] = {from_user_id: [[]]}

    # A queue of (previous user, next user) that we need to check.
    # Starts out with all pairs(from_user, friend_of_from_user).
    frontier = deque((from_user_id, friend_id)
                      for friend_id in friendships[from_user_id])
    
    # Keep going until we empty the queue
    while frontier:
      # Remove the pair that's next in the queue.
      prev_user_id, user_id = frontier.popleft()

      # Because of the way we're adding to the queue,
      # necessarily we already know some shortest paths to prev_user.
      paths_to_prev_user = shortest_paths_to[prev_user_id]
      new_paths_to_user = [path + [user_id] for path in paths_to_prev_user]

      # It's possible we already know a shortest path to user_id.
      old_paths_to_user = shortest_paths_to.get(user_id, [])

      # What's the shortest path to here that we've seen so far?
      if old_paths_to_user:
        min_path_length = len(old_paths_to_user[0])
      else: 
        min_path_length = float('inf')

      # Only keep paths that aren't too long and are actually new.
      new_paths_to_user = [path
                           for path in new_paths_to_user
                           if len(path) <= min_path_length
                           and path not in old_paths_to_user]
      
      shortest_paths_to[user_id] = old_paths_to_user + new_paths_to_user

      # Add never seen neighbors to the frontier.
      frontier.extend((user_id, friend_id)
                      for friend_id in friendships[user_id]
                      if friend_id not in shortest_paths_to)
  
    return shortest_paths_to

In [12]:
# For each from_user, for each to_user, a list of shortest paths
shortest_paths = {user.id: shortest_paths_from(user.id, friendships)
                    for user in users}

In [13]:
betweenness_centrality = {user.id: 0.0 for user in users}

for source in users:
  for target_id, paths in shortest_paths[source.id].items():
    if source.id < target_id: # don't double count
      num_paths = len(paths)
      contrib = 1 / num_paths
      for path in paths:
        for between_id in path:
          if between_id not in [source.id, target_id]:
            betweenness_centrality[between_id] += contrib

print(betweenness_centrality)

{0: 0.0, 1: 3.5, 2: 3.5, 3: 18.0, 4: 20.0, 5: 20.5, 6: 6.0, 7: 6.0, 8: 8.5, 9: 0.0}


In [14]:
# Closeness centrality
def farness(user_id: int) -> float:
  """The sum of the lengths of the shortest paths to each other user"""
  return sum(len(paths[0])
              for paths in shortest_paths[user_id].values())
  
closeness_centrality = {user.id: 1 / farness(user.id) for user in users}
print(closeness_centrality)

{0: 0.029411764705882353, 1: 0.037037037037037035, 2: 0.037037037037037035, 3: 0.045454545454545456, 4: 0.05, 5: 0.05, 6: 0.041666666666666664, 7: 0.041666666666666664, 8: 0.03571428571428571, 9: 0.027777777777777776}


In [17]:
# Computing shortest paths is kinda a pain, thus not used much on large networks. Usually, we use eigenvector centrality
from Chapter_04 import Matrix, make_matrix, shape

def matrix_times_matrix(m1: Matrix, m2: Matrix) -> Matrix:
  nr1, nc1 = shape(m1)
  nr2, nc2 = shape(m2)

  assert nc1 == nr2, "must have (# of columns in m1) == (# of rows in m2)"

  def entry_fn(i: int, j: int) -> float:
    """dot product of i-th row of m1 with j-th column of m2"""
    return sum(m1[i][k] * m2[k][j] for k in range(nc1))

  return make_matrix(nr1, nc2, entry_fn)

from Chapter_04 import Vector, dot

def matrix_times_vector(m: Matrix, v: Vector) -> Vector:
  nr, nc = shape(m)
  n = len(v)
  assert nc == n, "must hae (# cols in m) == (# elements in v)"

  return [dot(row, v) for row in m] # output has length nr

from typing import Tuple
import random
from Chapter_04 import magnitude, distance

def find_eigenvector(m: Matrix,
                     tolerance: float = 0.00001) -> Tuple[Vector, float]:
    guess = [random.random() for _ in m]

    while True:
      result = matrix_times_vector(m, guess) # transform guess
      norm = magnitude(result) # compute norm
      next_guess = [x / norm for x in result] # rescale

      if distance(guess, next_guess) < tolerance:
        # convergence so return (eigenvector, eigenvalue)
        return next_guess, norm
        
      guess = next_guess

def entry_fn(i: int, j: int):
  return 1 if (i,j) in friend_pairs or (j,i) in friend_pairs else 0

n = len(users)
adjacency_matrix = make_matrix(n,n, entry_fn)

[x,y] = find_eigenvector(adjacency_matrix)
print(x,y)


[0.3857811994691001, 0.5147899988028268, 0.5147899988028268, 0.4733142002995233, 0.2336069719947848, 0.15014841201517531, 0.08355056911204811, 0.08355056911204811, 0.07284246357716063, 0.02729222422304336] 2.6688229150782448
